In [85]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression

In [86]:
# Read in the data
data = pd.read_csv('Full_Data.csv', encoding = "ISO-8859-1")
data.head(1)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title


In [87]:
cutoff = '2015-01-01'
train = data[data['Date'] < cutoff]
test = data[data['Date'] >= cutoff]
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']
print(train.shape, test.shape, data.shape)

(3723, 27) (378, 27) (4101, 27)


In [88]:
# Removing punctuations
slicedData= train.iloc[:,2:27]
slicedData.replace(to_replace="[^a-zA-Z]", value=" ", regex=True, inplace=True)

# Renaming column names for ease of access
list1= [i for i in range(25)]
new_Index=[str(i) for i in list1]
slicedData.columns= new_Index
slicedData.head(5)

# Convertng headlines to lower case
for index in new_Index:
    slicedData[index]=slicedData[index].str.lower()
slicedData.head(1)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,a hindrance to operations extracts from the...,scorecard,hughes instant hit buoys blues,jack gets his skates on at ice cold alex,chaos as maracana builds up for united,depleted leicester prevail as elliott spoils e...,hungry spurs sense rich pickings,gunners so wide of an easy target,derby raise a glass to strupar s debut double,southgate strikes leeds pay the penalty,...,flintoff injury piles on woe for england,hunters threaten jospin with new battle of the...,kohl s successor drawn into scandal,the difference between men and women,sara denver nurse turned solicitor,diana s landmine crusade put tories in a panic,yeltsin s resignation caught opposition flat f...,russian roulette,sold out,recovering a title


In [89]:
headlines = []
for row in range(0,len(slicedData.index)):
    headlines.append(' '.join(str(x) for x in slicedData.iloc[row,0:25]))
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))

In [90]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score 

def nn(alpha, hidden, gram):
    basicvectorizer = CountVectorizer(ngram_range=(gram,gram))
    basictrain = basicvectorizer.fit_transform(headlines)
    basicmodel = MLPClassifier(solver='lbfgs', alpha=alpha, hidden_layer_sizes=hidden, random_state=1)
    basicmodel = basicmodel.fit(basictrain, train["Label"])
    
    basictest = basicvectorizer.transform(testheadlines)
    predictions = basicmodel.predict(basictest)
#     return pd.crosstab(test["Label"], predictions, rownames=["Actual"], colnames=["Predicted"])
    return accuracy_score(test["Label"], predictions)
    

In [91]:

print(nn(1e-5, (10,10), 2))

0.5634920634920635


In [92]:
# alphas = [1, 0.1, 0.01, 0.001, 0.0001]
alphas = [0.0001]
# hidden = [(1, 1), (10, 1), (1, 10), (10, 10), (100, 100)]
hidden = [(10, 10)]
grams = [1,2,3,4]
for a in alphas:
    for h in hidden:
        for g in grams:
            print(a, h, g, nn(a, h, g))

0.0001 (10, 10) 1 0.47619047619047616
0.0001 (10, 10) 2 0.5608465608465608
0.0001 (10, 10) 3 0.5158730158730159
0.0001 (10, 10) 4 0.5211640211640212
